In [1]:

APELLIDO = "Relloso"
NOMBRE = "Daniel"
REPO_NAME = f"Práctica 3: El Desafío de Visión Profunda (Deep Vision)_{APELLIDO}{NOMBRE}"

import os, random, json, datetime, hashlib
from pathlib import Path

import numpy as np
import tensorflow as tf

# --- Fijar semillas (Python / NumPy / TF) ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Crear estructura mínima del repo (en Colab: carpeta local) ---
base_dir = Path(REPO_NAME)
notebooks_dir = base_dir / "notebooks"
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
outputs_dir = base_dir / "outputs"
env_dir = base_dir / "env"

for d in [base_dir, notebooks_dir, results_dir, figuras_dir, outputs_dir, env_dir]:
    d.mkdir(parents=True, exist_ok=True)

# README y .gitignore mínimos (si no existen)
readme_path = base_dir / "README.md"
if not readme_path.exists():
    readme_path.write_text("# Proyecto CIFAR-10 CNN\n\nPráctica de IA sobre CIFAR-10.\n")

gitignore_path = base_dir / ".gitignore"
if not gitignore_path.exists():
    gitignore_path.write_text("""__pycache__/
.ipynb_checkpoints/
env/
outputs/
results/
""")

print("Estructura de repo creada en:", base_dir.resolve())


Estructura de repo creada en: /content/Práctica 3: El Desafío de Visión Profunda (Deep Vision)_RellosoDaniel


In [ ]:
import platform
import pkg_resources

env_md_path = env_dir / "ENVIRONMENT.md"
req_path = env_dir / "requirements.txt"

# Info de versiones básicas
python_version = platform.python_version()
tf_version = tf.__version__

# Info GPU (si existe)
gpus = tf.config.list_physical_devices('GPU')
gpu_info = "GPU disponible: " + (gpus[0].name if gpus else "No")

env_text = f"""# Entorno de ejecución

- Python: {python_version}
- TensorFlow: {tf_version}
- {gpu_info}
"""

env_md_path.write_text(env_text)

# Congelar dependencias principales del entorno actual
installed_packages = sorted(
    [f"{d.project_name}=={d.version}" for d in pkg_resources.working_set]
)

req_path.write_text("\n".join(installed_packages))

print("ENVIRONMENT.md y requirements.txt creados en env/")


In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

print("x_train_full shape:", x_train_full.shape)
print("y_train_full shape:", y_train_full.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split

# y_train_full está en forma (50000,1) -> lo aplano a vector 1D
y_flat = y_train_full.ravel()

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full,
    y_flat,
    test_size=0.2,
    random_state=SEED,
    stratify=y_flat
)

print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_valid:", x_valid.shape)
print("y_valid:", y_valid.shape)


In [ ]:
from tensorflow.keras.utils import to_categorical

# Normalizar a [0,1]
x_train = x_train.astype("float32") / 255.0
x_valid = x_valid.astype("float32") / 255.0
x_test  = x_test.astype("float32")  / 255.0

# One-hot
NUM_CLASSES = 10
y_train_oh = to_categorical(y_train, NUM_CLASSES)
y_valid_oh = to_categorical(y_valid, NUM_CLASSES)
y_test_oh  = to_categorical(y_test, NUM_CLASSES)

print("x_train min/max:", x_train.min(), x_train.max())
print("y_train_oh shape:", y_train_oh.shape)
print("y_valid_oh shape:", y_valid_oh.shape)
print("y_test_oh shape:", y_test_oh.shape)


In [ ]:
import matplotlib.pyplot as plt

class_names = [
    "airplane", "automobile", "bird", "cat",
    "deer", "dog", "frog", "horse",
    "ship", "truck"
]

plt.figure(figsize=(6,6))
for i in range(16):
    idx = i  # puedes mezclar si quieres: np.random.randint(0, x_train.shape[0])
    plt.subplot(4, 4, i+1)
    plt.imshow(x_train[idx])
    plt.axis("off")
    plt.title(class_names[y_train[idx]])
plt.tight_layout()
plt.show()


In [ ]:
import yaml

# --- R4: data_meta.json con formas, fecha/hora y hash de primeras 1024 imágenes ---

def compute_data_hash(x, n_samples=1024):
    subset = x[:n_samples]
    # Convertimos a bytes (tras normalización)
    data_bytes = subset.tobytes()
    h = hashlib.sha256(data_bytes).hexdigest()
    return h

data_meta = {
    "shapes": {
        "x_train": list(x_train.shape),
        "x_valid": list(x_valid.shape),
        "x_test":  list(x_test.shape),
        "y_train_oh": list(y_train_oh.shape),
        "y_valid_oh": list(y_valid_oh.shape),
        "y_test_oh":  list(y_test_oh.shape),
    },
    "datetime": datetime.datetime.now().isoformat(),
    "hash": compute_data_hash(x_train, n_samples=1024)
}

data_meta_path = results_dir / "data_meta.json"
with data_meta_path.open("w") as f:
    json.dump(data_meta, f, indent=4)

print("Guardado:", data_meta_path)

# --- R3: params.yaml con config inicial del modelo/entrenamiento ---

params = {
    "model": {
        "blocks": 2,
        "filters": [32, 64],
        "kernel_size": 3,
        "dense_units": 128,
        "dropout": 0.5
    },
    "training": {
        "lr": 1e-3,
        "batch_size": 64,
        "epochs": 30,
        "augment": False,
        "seed": SEED
    }
}

params_path = results_dir / "params.yaml"
with params_path.open("w") as f:
    yaml.dump(params, f, sort_keys=False)

print("Guardado:", params_path)


In [ ]:
def smoke_test_model():
    try:
        from tensorflow.keras import layers, models

        model = models.Sequential([
            layers.Input(shape=(32, 32, 3)),
            layers.Conv2D(8, (3, 3), activation="relu"),
            layers.Flatten(),
            layers.Dense(10, activation="softmax")
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        print("Smoke test OK: modelo construido y compilado.")
        return True
    except Exception as e:
        print("Smoke test FAILED:", e)
        return False

smoke_test_model()


In [ ]:
print("x_train:", x_train.shape)
print("x_valid:", x_valid.shape)
print("x_test:", x_test.shape)

print("y_train_oh:", y_train_oh.shape)
print("y_valid_oh:", y_valid_oh.shape)
print("y_test_oh:", y_test_oh.shape)

print("data_meta.json existe:", data_meta_path.exists())
print("params.yaml existe:", params_path.exists())


¿Por qué normalizar /255?
Porque los píxeles vienen en el rango [0,255] y son valores relativamente grandes y desbalanceados para las capas iniciales. Al dividir entre 255 llevamos todo a [0,1], lo que hace que las activaciones y los gradientes se mantengan en rangos más estables, permite usar tasas de aprendizaje más razonables y facilita que el optimizador encuentre una buena solución.

¿Por qué estratificar aquí y no al final?
Porque quiero que desde el principio el split train/valid mantenga la misma proporción de clases que el conjunto original. Si estratifico sobre y_train_full antes de cualquier otro procesamiento, me aseguro de que tanto x_train como x_valid sean representativos del problema real. Si hiciera el split al final, después de alguna transformación rara (por ejemplo, filtrar clases o reequilibrar), podría romper esa representatividad o acabar con un conjunto de validación pobremente balanceado, lo que sesgaría la evaluación del modelo.

In [ ]:
import os, json, datetime
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import pandas as pd
import yaml

# === Ajusta estos nombres igual que en el PROMPT 1 ===
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# === Definición del MLP baseline ===
mlp_model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])

mlp_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Mostrar y guardar summary (para trazabilidad)
summary_path = results_dir / "mlp_summary.txt"
with open(summary_path, "w") as f:
    mlp_model.summary(print_fn=lambda x: f.write(x + "\n"))

mlp_model.summary()
print("Resumen guardado en:", summary_path)

# === Entrenamiento 10 épocas, batch 64, con validación ===
EPOCHS = 10
BATCH_SIZE = 64

mlp_history = mlp_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)


In [ ]:
# === Evaluación en validación y test ===
val_loss, val_acc = mlp_model.evaluate(x_valid, y_valid_oh, verbose=0)
test_loss, test_acc = mlp_model.evaluate(x_test, y_test_oh, verbose=0)
print(f"Validación - loss: {val_loss:.4f}, acc: {val_acc:.4f}")
print(f"Test       - loss: {test_loss:.4f}, acc: {test_acc:.4f}")

# === Curvas de entrenamiento ===
history_dict = mlp_history.history

plt.figure(figsize=(10,4))

# Loss
plt.subplot(1,2,1)
plt.plot(history_dict["loss"], label="train_loss")
plt.plot(history_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss MLP")
plt.legend()

# Accuracy
plt.subplot(1,2,2)
plt.plot(history_dict["accuracy"], label="train_acc")
plt.plot(history_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy MLP")
plt.legend()

plt.tight_layout()

# Nombre de fichero con fecha + commit corto
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d")

# Cambia este string por el hash corto real de tu commit
commit_short = "ab12cd"

fig_name = f"{timestamp}_{commit_short}_mlp_curvas.png"
fig_path = figuras_dir / fig_name
plt.savefig(fig_path, dpi=150)
plt.show()

print("Figura guardada en:", fig_path)


In [ ]:
# === Exportar history a CSV ===
history_df = pd.DataFrame(history_dict)
history_csv_path = results_dir / "history.csv"
history_df.to_csv(history_csv_path, index=False)
print("History guardado en:", history_csv_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

# Añadimos o actualizamos la sección del MLP baseline
params["mlp_baseline"] = {
    "architecture": "Input(32,32,3) -> Flatten -> Dense(256,relu) -> Dropout(0.5) -> Dense(10,softmax)",
    "hidden_units": 256,
    "dropout": 0.5,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "seed": 42
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado en:", params_path)


In [ ]:
metrics_path = results_dir / "metrics.json"

current_metrics = {
    "model": "mlp_baseline",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(val_loss),
    "val_acc": float(val_acc),
    "test_loss": float(test_loss),
    "test_acc": float(test_acc),
    "commit": commit_short
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(current_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado en:", metrics_path)


In [ ]:
print("Resumen guardado:", summary_path.exists())
print("History CSV:", history_csv_path.exists())
print("Metrics JSON:", metrics_path.exists())
print("Figura curvas:", fig_path.exists())


¿Hay overfitting?
En las curvas se ve que la loss de entrenamiento sigue bajando y la accuracy de train sube más que la de validación. A partir de unas pocas épocas, la loss de validación se estabiliza o incluso empeora ligeramente mientras el modelo sigue mejorando en train. Esto indica un cierto overfitting: el MLP empieza a memorizar patrones específicos del conjunto de entrenamiento y deja de mejorar en datos nuevos.

¿Por qué el aplanado limita la generalización?
Al aplanar la imagen 32×32×3 a un vector de 3072 valores, el modelo pierde toda la estructura espacial: ya no sabe qué píxeles son vecinos ni puede explotar patrones locales (bordes, texturas, partes del objeto). Cada neurona densa ve la imagen como una lista de números sin geometría. Eso obliga al MLP a “aprender desde cero” relaciones que una CNN incorpora de forma natural mediante convoluciones y peso compartido. El resultado es un modelo con muchos parámetros, más sensible al ruido de fondo y con peor capacidad de generalizar en imágenes que una CNN diseñada para explotar la estructura espacial.

In [ ]:
import time
from tensorflow.keras import layers, models
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import json, yaml
from datetime import datetime

# Mismos nombres que antes
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "DAniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# === Definir CNN de 2 bloques ===
cnn_model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])

cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Guardar summary para trazabilidad
cnn_summary_path = results_dir / "cnn_2blocks_summary.txt"
with open(cnn_summary_path, "w") as f:
    cnn_model.summary(print_fn=lambda x: f.write(x + "\n"))

cnn_model.summary()
print("Resumen CNN guardado en:", cnn_summary_path)

# === Entrenar 15 épocas, batch=64, con validación y temporizando ===
EPOCHS_CNN = 15
BATCH_SIZE = 64

start_time = time.time()
cnn_history = cnn_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS_CNN,
    batch_size=BATCH_SIZE,
    verbose=1
)
total_time = time.time() - start_time
time_per_epoch_cnn = total_time / EPOCHS_CNN
print(f"Tiempo total entrenamiento CNN: {total_time:.2f} s")
print(f"Tiempo medio por época CNN: {time_per_epoch_cnn:.2f} s")


In [ ]:
# Evaluación en validación y test
cnn_val_loss, cnn_val_acc = cnn_model.evaluate(x_valid, y_valid_oh, verbose=0)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(x_test, y_test_oh, verbose=0)
print(f"Validación CNN - loss: {cnn_val_loss:.4f}, acc: {cnn_val_acc:.4f}")
print(f"Test CNN       - loss: {cnn_test_loss:.4f}, acc: {cnn_test_acc:.4f}")

cnn_history_dict = cnn_history.history

plt.figure(figsize=(10,4))

# Loss
plt.subplot(1,2,1)
plt.plot(cnn_history_dict["loss"], label="train_loss")
plt.plot(cnn_history_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss CNN 2-block")
plt.legend()

# Accuracy
plt.subplot(1,2,2)
plt.plot(cnn_history_dict["accuracy"], label="train_acc")
plt.plot(cnn_history_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy CNN 2-block")
plt.legend()

plt.tight_layout()

timestamp = datetime.now().strftime("%Y-%m-%d")
commit_short_cnn = "ef34gh"  # cambia esto por el hash corto real

fig_name_cnn = f"{timestamp}_{commit_short_cnn}_cnn2blocks_curvas.png"
fig_path_cnn = figuras_dir / fig_name_cnn
plt.savefig(fig_path_cnn, dpi=150)
plt.show()

print("Figura CNN guardada en:", fig_path_cnn)


In [ ]:
cnn_history_df = pd.DataFrame(cnn_history_dict)
cnn_history_csv_path = results_dir / "history_cnn_2blocks.csv"
cnn_history_df.to_csv(cnn_history_csv_path, index=False)
print("History CNN guardado en:", cnn_history_csv_path)


In [ ]:
metrics_path = results_dir / "metrics.json"

cnn_metrics = {
    "model": "cnn_2blocks",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(cnn_val_loss),
    "val_acc": float(cnn_val_acc),
    "test_loss": float(cnn_test_loss),
    "test_acc": float(cnn_test_acc),
    "time_per_epoch": float(time_per_epoch_cnn),
    "params": int(cnn_model.count_params()),
    "commit": commit_short_cnn
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(cnn_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con la CNN:", metrics_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["cnn_2blocks"] = {
    "blocks": 2,
    "filters": [32, 64],
    "kernel_size": 3,
    "dense_units": 128,
    "dropout": 0.5,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS_CNN,
    "seed": 42
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con la CNN en:", params_path)


In [ ]:
# Define o reutiliza estas variables del MLP:
# - mlp_model (del prompt 2)
# - val_acc, test_acc del MLP (puedes renombrarlos para evitar líos)
# - time_per_epoch_mlp (mídelo con un timer, o pon el valor observado a mano)

mlp_params = mlp_model.count_params()

# Si guardaste las métricas del MLP en variables diferentes, ajústalo:
val_acc_mlp  = float(val_acc)      # renombra si hace falta
test_acc_mlp = float(test_acc)

# Rellena esto con el valor real que observes
time_per_epoch_mlp = 0.0  # sustituye por el tiempo medio real del MLP

comparacion_df = pd.DataFrame([
    {
        "modelo": "MLP baseline",
        "params": mlp_params,
        "time_per_epoch_s": time_per_epoch_mlp,
        "val_acc": val_acc_mlp,
        "test_acc": test_acc_mlp
    },
    {
        "modelo": "CNN 2-blocks",
        "params": cnn_model.count_params(),
        "time_per_epoch_s": time_per_epoch_cnn,
        "val_acc": float(cnn_val_acc),
        "test_acc": float(cnn_test_acc)
    }
])

comparacion_path = results_dir / "comparacion_mlp_vs_cnn.csv"
comparacion_df.to_csv(comparacion_path, index=False)

print(comparacion_df)
print("Tabla comparativa guardada en:", comparacion_path)


In [ ]:
print("Summary CNN guardado:", cnn_summary_path.exists())
print("History CNN:", cnn_history_csv_path.exists())
print("Metrics JSON:", metrics_path.exists())
print("Figura CNN:", fig_path_cnn.exists())
print("Tabla comparativa:", comparacion_path.exists())


¿Por qué la CNN puede rendir mejor con igual o menos parámetros?
La CNN introduce un sesgo inductivo espacial: asume que las imágenes tienen estructura local y que los mismos patrones (bordes, texturas, partes de objetos) se repiten en distintas posiciones. Las capas convolucionales usan peso compartido: un mismo filtro se aplica en toda la imagen, lo que reduce mucho el número de parámetros frente a una capa densa que conecta cada píxel con cada neurona. Además, el pooling hace la representación más robusta a pequeñas traslaciones y ruido de fondo. Todo esto permite que la CNN “gaste” sus parámetros en capturar patrones visuales relevantes en vez de memorizar píxeles concretos, lo que suele dar mejor generalización en CIFAR-10 incluso con un número de parámetros similar o menor que el MLP baseline.


In [ ]:
from pathlib import Path
from datetime import datetime
import time, json, yaml
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

# Mismos nombres de repo que antes
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# === Definir CNN 2 bloques con L2 ===
weight_decay = 1e-4

cnn_l2_model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(
        32, (3, 3), activation="relu", padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    ),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(
        64, (3, 3), activation="relu", padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    ),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(
        128, activation="relu",
        kernel_regularizer=regularizers.l2(weight_decay)
    ),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])

cnn_l2_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Guardar summary
cnn_l2_summary_path = results_dir / "cnn_2blocks_l2_es_summary.txt"
with open(cnn_l2_summary_path, "w") as f:
    cnn_l2_model.summary(print_fn=lambda x: f.write(x + "\n"))

cnn_l2_model.summary()
print("Resumen CNN L2+ES guardado en:", cnn_l2_summary_path)


In [ ]:
EPOCHS_L2 = 30
BATCH_SIZE = 64

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

start_time = time.time()
cnn_l2_history = cnn_l2_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS_L2,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop],
    verbose=1
)
total_time_l2 = time.time() - start_time
time_per_epoch_l2 = total_time_l2 / len(cnn_l2_history.history["loss"])

print(f"Épocas realmente entrenadas: {len(cnn_l2_history.history['loss'])}")
print(f"Tiempo total: {total_time_l2:.2f} s")
print(f"Tiempo medio por época: {time_per_epoch_l2:.2f} s")


In [ ]:
cnn_l2_val_loss, cnn_l2_val_acc = cnn_l2_model.evaluate(x_valid, y_valid_oh, verbose=0)
cnn_l2_test_loss, cnn_l2_test_acc = cnn_l2_model.evaluate(x_test, y_test_oh, verbose=0)

print(f"Validación CNN L2+ES - loss: {cnn_l2_val_loss:.4f}, acc: {cnn_l2_val_acc:.4f}")
print(f"Test CNN L2+ES       - loss: {cnn_l2_test_loss:.4f}, acc: {cnn_l2_test_acc:.4f}")


In [ ]:
cnn_l2_hist_dict = cnn_l2_history.history

plt.figure(figsize=(10,4))

# Loss
plt.subplot(1,2,1)
plt.plot(cnn_l2_hist_dict["loss"], label="train_loss")
plt.plot(cnn_l2_hist_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss CNN 2-blocks L2 + ES")
plt.legend()

# Accuracy
plt.subplot(1,2,2)
plt.plot(cnn_l2_hist_dict["accuracy"], label="train_acc")
plt.plot(cnn_l2_hist_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy CNN 2-blocks L2 + ES")
plt.legend()

plt.tight_layout()

timestamp = datetime.now().strftime("%Y-%m-%d")
commit_short_l2 = "jk78lm"  # pon aquí el hash corto real del commit

fig_name_l2 = f"{timestamp}_{commit_short_l2}_cnn2blocks_l2_es_curvas.png"
fig_path_l2 = figuras_dir / fig_name_l2
plt.savefig(fig_path_l2, dpi=150)
plt.show()

print("Figura CNN L2+ES guardada en:", fig_path_l2)


In [ ]:
cnn_l2_history_df = pd.DataFrame(cnn_l2_hist_dict)
cnn_l2_history_csv_path = results_dir / "history_cnn_2blocks_l2_es.csv"
cnn_l2_history_df.to_csv(cnn_l2_history_csv_path, index=False)
print("History CNN L2+ES guardado en:", cnn_l2_history_csv_path)


In [ ]:
metrics_path = results_dir / "metrics.json"

cnn_l2_metrics = {
    "model": "cnn_2blocks_l2_es",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(cnn_l2_val_loss),
    "val_acc": float(cnn_l2_val_acc),
    "test_loss": float(cnn_l2_test_loss),
    "test_acc": float(cnn_l2_test_acc),
    "time_per_epoch": float(time_per_epoch_l2),
    "params": int(cnn_l2_model.count_params()),
    "l2": float(weight_decay),
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True,
        "epochs_trained": len(cnn_l2_hist_dict["loss"])
    },
    "commit": commit_short_l2
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(cnn_l2_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con CNN L2+ES:", metrics_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["cnn_2blocks_l2_es"] = {
    "blocks": 2,
    "filters": [32, 64],
    "kernel_size": 3,
    "dense_units": 128,
    "dropout": 0.5,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "batch_size": BATCH_SIZE,
    "max_epochs": EPOCHS_L2,
    "seed": 42,
    "l2": weight_decay,
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True
    }
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con CNN L2+ES en:", params_path)


In [ ]:
print("Resumen CNN L2+ES:", cnn_l2_summary_path.exists())
print("History L2+ES CSV:", cnn_l2_history_csv_path.exists())
print("Metrics JSON:", metrics_path.exists())
print("Figura L2+ES:", fig_path_l2.exists())

print("Épocas entrenadas (debería ser <= 30):", len(cnn_l2_hist_dict["loss"]))
print("Mínima val_loss alcanzada:", min(cnn_l2_hist_dict["val_loss"]))


¿Se redujo la brecha train/val?
Con la regularización L2 y el EarlyStopping, la curva de loss de entrenamiento ya no se separa tanto de la loss de validación. El modelo deja de entrenarse cuando val_loss deja de mejorar, de modo que no sigue “bajando” solo en train mientras empeora en valid. En las curvas se ve que la diferencia entre train_loss y val_loss es más pequeña que en la CNN sin L2, lo que indica una brecha train/val reducida y menos sobreajuste.

¿Subió la test accuracy?
Aunque la accuracy de entrenamiento suele ser algo menor (el modelo está más “limitado” por la penalización L2 y la parada temprana), la test accuracy suele mantenerse o mejorar ligeramente respecto a la CNN sin regularización fuerte. Si la test_acc ha subido, es señal de que el modelo generaliza mejor. Si se mantiene muy parecida pero con una brecha train/val más pequeña, también es un buen resultado: se ha ganado estabilidad y robustez sin perder rendimiento en test. En cualquier caso, lo importante es que el modelo ya no se “dispara” en train mientras se degrada en valid, lo que refleja una generalización más sana.

In [ ]:
from pathlib import Path
from datetime import datetime
import time, json, yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Mismos nombres que antes
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# --- Data Augmentation moderado ---
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),          # rotación ligera (±10% de vuelta)
    layers.RandomZoom(0.10),              # zoom leve
    layers.RandomTranslation(0.10, 0.10)  # traslación leve
], name="data_augmentation")

weight_decay = 1e-4
BATCH_SIZE = 64
EPOCHS_AUG = 20

# --- CNN 2 blocks + L2 + Augment ---
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)

x = layers.Conv2D


In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

# Callback sencillo para registrar LR por época
class LrLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.lrs = []

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        self.lrs.append(lr)

lr_logger = LrLogger()


In [ ]:
start_time = time.time()
cnn_aug_history = cnn_aug_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS_AUG,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, reduce_lr, lr_logger],
    verbose=1
)
total_time_aug = time.time() - start_time
time_per_epoch_aug = total_time_aug / len(cnn_aug_history.history["loss"])

print(f"Épocas entrenadas (máx 20): {len(cnn_aug_history.history['loss'])}")
print(f"Tiempo total: {total_time_aug:.2f} s")
print(f"Tiempo medio por época: {time_per_epoch_aug:.2f} s")


In [ ]:
cnn_aug_val_loss, cnn_aug_val_acc = cnn_aug_model.evaluate(x_valid, y_valid_oh, verbose=0)
cnn_aug_test_loss, cnn_aug_test_acc = cnn_aug_model.evaluate(x_test, y_test_oh, verbose=0)

print(f"Validación CNN Aug+L2 - loss: {cnn_aug_val_loss:.4f}, acc: {cnn_aug_val_acc:.4f}")
print(f"Test CNN Aug+L2       - loss: {cnn_aug_test_loss:.4f}, acc: {cnn_aug_test_acc:.4f}")

cnn_aug_hist_dict = cnn_aug_history.history

timestamp = datetime.now().strftime("%Y-%m-%d")
commit_short_aug = "rs90ab"  # pon aquí tu hash corto real

# --- Curvas loss/acc ---
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(cnn_aug_hist_dict["loss"], label="train_loss")
plt.plot(cnn_aug_hist_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss CNN 2-blocks Aug+L2+ES+RLROP")
plt.legend()

plt.subplot(1,2,2)
plt.plot(cnn_aug_hist_dict["accuracy"], label="train_acc")
plt.plot(cnn_aug_hist_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy CNN 2-blocks Aug+L2+ES+RLROP")
plt.legend()

plt.tight_layout()

fig_name_aug = f"{timestamp}_{commit_short_aug}_cnn2blocks_aug_curvas.png"
fig_path_aug = figuras_dir / fig_name_aug
plt.savefig(fig_path_aug, dpi=150)
plt.show()

print("Figura curvas CNN Aug guardada en:", fig_path_aug)

# --- Curva de LR ---
plt.figure(figsize=(5,4))
plt.plot(lr_logger.lrs, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Learning rate")
plt.title("Evolución del LR (ReduceLROnPlateau)")
plt.tight_layout()

fig_lr_name = f"{timestamp}_{commit_short_aug}_lr_schedule.png"
fig_lr_path = figuras_dir / fig_lr_name
plt.savefig(fig_lr_path, dpi=150)
plt.show()

print("Figura LR guardada en:", fig_lr_path)


In [ ]:
# Elegimos una imagen de ejemplo
idx = 0
sample_img = x_train[idx]  # ya está en [0,1]

plt.figure(figsize=(6,6))
for i in range(9):
    augmented = data_augmentation(
        tf.expand_dims(sample_img, axis=0),
        training=True
    )
    aug_img = augmented[0].numpy().clip(0,1)

    plt.subplot(3,3,i+1)
    plt.imshow(aug_img)
    plt.axis("off")

plt.tight_layout()

fig_aug_samples_name = f"{timestamp}_{commit_short_aug}_aug_samples.png"
fig_aug_samples_path = figuras_dir / fig_aug_samples_name
plt.savefig(fig_aug_samples_path, dpi=150)
plt.show()

print("Figura de muestras de augmentation guardada en:", fig_aug_samples_path)


In [ ]:
# History
cnn_aug_history_df = pd.DataFrame(cnn_aug_hist_dict)
cnn_aug_history_df["lr"] = lr_logger.lrs  # guardamos también LR por época

cnn_aug_history_csv_path = results_dir / "history_cnn_2blocks_aug_l2_es_rlrop.csv"
cnn_aug_history_df.to_csv(cnn_aug_history_csv_path, index=False)
print("History CNN Aug guardado en:", cnn_aug_history_csv_path)

# Métricas
metrics_path = results_dir / "metrics.json"

cnn_aug_metrics = {
    "model": "cnn_2blocks_l2_aug_rlrop",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(cnn_aug_val_loss),
    "val_acc": float(cnn_aug_val_acc),
    "test_loss": float(cnn_aug_test_loss),
    "test_acc": float(cnn_aug_test_acc),
    "time_per_epoch": float(time_per_epoch_aug),
    "params": int(cnn_aug_model.count_params()),
    "l2": float(weight_decay),
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "scheduler": {
        "type": "ReduceLROnPlateau",
        "monitor": "val_loss",
        "factor": 0.2,
        "patience": 3,
        "min_lr": 1e-6
    },
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True,
        "epochs_trained": len(cnn_aug_hist_dict["loss"])
    },
    "commit": commit_short_aug
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(cnn_aug_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con CNN Aug:", metrics_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["cnn_2blocks_l2_aug_rlrop"] = {
    "blocks": 2,
    "filters": [32, 64],
    "kernel_size": 3,
    "dense_units": 128,
    "dropout": 0.5,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "batch_size": BATCH_SIZE,
    "max_epochs": EPOCHS_AUG,
    "seed": 42,
    "l2": weight_decay,
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "scheduler": "ReduceLROnPlateau",
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True
    }
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con CNN Aug+RLROP en:", params_path)


In [ ]:
print("Resumen CNN Aug:", cnn_aug_summary_path.exists())
print("History Aug CSV:", cnn_aug_history_csv_path.exists())
print("Metrics JSON:", metrics_path.exists())
print("Figura curvas:", fig_path_aug.exists())
print("Figura LR:", fig_lr_path.exists())
print("Figura muestras augmentation:", fig_aug_samples_path.exists())

print("Épocas entrenadas (<=20):", len(cnn_aug_hist_dict["loss"]))
print("Mínima val_loss:", min(cnn_aug_hist_dict["val_loss"]))
print("LRs por época:", lr_logger.lrs)


¿Mejoró la test accuracy?
Con el Data Augmentation moderado y el scheduler ReduceLROnPlateau, la test_acc suele subir algo respecto al modelo con solo L2+EarlyStopping, especialmente si el dataset original era algo justo para la capacidad de la red. El augmentation genera variantes razonables de las imágenes (flips, rotaciones pequeñas, zoom y traslación), lo que fuerza al modelo a aprender características más invariantes y menos dependientes de la posición exacta o del ruido específico del conjunto de entrenamiento.
¿Cómo afectó a la convergencia?
El entrenamiento se vuelve algo más ruidoso y lento por época (se aplican transformaciones en cada batch), y las curvas de loss/accuracy convergen de forma más suave. ReduceLROnPlateau baja la tasa de aprendizaje cuando val_loss se estanca, lo que se ve en la curva de LR: tras varios epochs sin mejora clara, el LR cae y la loss de validación puede seguir refinándose. En resumen, la convergencia es menos “rápida” al principio, pero más estable al final, y el modelo final tiende a generalizar mejor aunque tarde algún epoch extra en llegar a su mejor punto.


In [ ]:
from pathlib import Path
from datetime import datetime
import time, json, yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Mismos identificadores que en los prompts anteriores
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# Reutilizamos un augment moderado (puedes mantener el del prompt 5)
data_augmentation_3b = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.10, 0.10)
], name="data_augmentation_3blocks")

weight_decay = 1e-4
BATCH_SIZE = 64
EPOCHS_3BLOCKS = 30

# --- CNN 3 bloques: 32 -> 64 -> 128 ---
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation_3b(inputs)

# Bloque 1: 32 filtros
x = layers.Conv2D(
    32, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Bloque 2: 64 filtros
x = layers.Conv2D(
    64, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Bloque 3: 128 filtros
x = layers.Conv2D(
    128, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Cabeza densa
x = layers.Flatten()(x)
x = layers.Dense(
    128, activation="relu",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)

cnn_3b_model = models.Model(inputs=inputs, outputs=outputs, name="cnn_3blocks_l2_aug")

cnn_3b_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Guardar summary
cnn_3b_summary_path = results_dir / "cnn_3blocks_l2_aug_rlrop_summary.txt"
with open(cnn_3b_summary_path, "w") as f:
    cnn_3b_model.summary(print_fn=lambda l: f.write(l + "\n"))

cnn_3b_model.summary()
print("Resumen CNN 3-blocks guardado en:", cnn_3b_summary_path)
print("Parámetros CNN 3-blocks:", cnn_3b_model.count_params())


In [ ]:
early_stop_3b = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_3b = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

class LrLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.lrs = []

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        self.lrs.append(lr)

lr_logger_3b = LrLogger()


In [ ]:
start_time_3b = time.time()
cnn_3b_history = cnn_3b_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS_3BLOCKS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop_3b, reduce_lr_3b, lr_logger_3b],
    verbose=1
)
total_time_3b = time.time() - start_time_3b
time_per_epoch_3b = total_time_3b / len(cnn_3b_history.history["loss"])

print(f"Épocas entrenadas (máx 30): {len(cnn_3b_history.history['loss'])}")
print(f"Tiempo total 3-blocks: {total_time_3b:.2f} s")
print(f"Tiempo medio por época 3-blocks: {time_per_epoch_3b:.2f} s")


In [ ]:
cnn_3b_val_loss, cnn_3b_val_acc = cnn_3b_model.evaluate(x_valid, y_valid_oh, verbose=0)
cnn_3b_test_loss, cnn_3b_test_acc = cnn_3b_model.evaluate(x_test, y_test_oh, verbose=0)

print(f"Validación CNN 3-blocks - loss: {cnn_3b_val_loss:.4f}, acc: {cnn_3b_val_acc:.4f}")
print(f"Test CNN 3-blocks       - loss: {cnn_3b_test_loss:.4f}, acc: {cnn_3b_test_acc:.4f}")

cnn_3b_hist_dict = cnn_3b_history.history

timestamp = datetime.now().strftime("%Y-%m-%d")
commit_short_3b = "tu12h3"  # pon aquí el hash corto real del commit

# --- Curvas loss/acc ---
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(cnn_3b_hist_dict["loss"], label="train_loss")
plt.plot(cnn_3b_hist_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss CNN 3-blocks L2+Aug+ES+RLROP")
plt.legend()

plt.subplot(1,2,2)
plt.plot(cnn_3b_hist_dict["accuracy"], label="train_acc")
plt.plot(cnn_3b_hist_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy CNN 3-blocks L2+Aug+ES+RLROP")
plt.legend()

plt.tight_layout()

fig_name_3b = f"{timestamp}_{commit_short_3b}_cnn3blocks_curvas.png"
fig_path_3b = figuras_dir / fig_name_3b
plt.savefig(fig_path_3b, dpi=150)
plt.show()

print("Figura curvas CNN 3-blocks guardada en:", fig_path_3b)

# --- Curva de LR ---
plt.figure(figsize=(5,4))
plt.plot(lr_logger_3b.lrs, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Learning rate")
plt.title("LR schedule CNN 3-blocks")
plt.tight_layout()

fig_lr_3b_name = f"{timestamp}_{commit_short_3b}_cnn3blocks_lr.png"
fig_lr_3b_path = figuras_dir / fig_lr_3b_name
plt.savefig(fig_lr_3b_path, dpi=150)
plt.show()

print("Figura LR CNN 3-blocks guardada en:", fig_lr_3b_path)


In [ ]:
# History
cnn_3b_history_df = pd.DataFrame(cnn_3b_hist_dict)
cnn_3b_history_df["lr"] = lr_logger_3b.lrs

cnn_3b_history_csv_path = results_dir / "history_cnn_3blocks_l2_aug_rlrop.csv"
cnn_3b_history_df.to_csv(cnn_3b_history_csv_path, index=False)
print("History CNN 3-blocks guardado en:", cnn_3b_history_csv_path)

# Métricas
metrics_path = results_dir / "metrics.json"

cnn_3b_metrics = {
    "model": "cnn_3blocks_l2_aug_rlrop",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(cnn_3b_val_loss),
    "val_acc": float(cnn_3b_val_acc),
    "test_loss": float(cnn_3b_test_loss),
    "test_acc": float(cnn_3b_test_acc),
    "time_per_epoch": float(time_per_epoch_3b),
    "params": int(cnn_3b_model.count_params()),
    "l2": float(weight_decay),
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "scheduler": {
        "type": "ReduceLROnPlateau",
        "monitor": "val_loss",
        "factor": 0.2,
        "patience": 3,
        "min_lr": 1e-6
    },
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True,
        "epochs_trained": len(cnn_3b_hist_dict["loss"])
    },
    "commit": commit_short_3b
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(cnn_3b_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con CNN 3-blocks:", metrics_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["cnn_3blocks_l2_aug_rlrop"] = {
    "blocks": 3,
    "filters": [32, 64, 128],
    "kernel_size": 3,
    "dense_units": 128,
    "dropout": 0.5,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "batch_size": BATCH_SIZE,
    "max_epochs": EPOCHS_3BLOCKS,
    "seed": 42,
    "l2": weight_decay,
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "scheduler": "ReduceLROnPlateau",
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True
    }
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con CNN 3-blocks en:", params_path)


In [ ]:
comparacion_path = results_dir / "comparacion_mlp_vs_cnn.csv"

# Leemos la tabla si existe
if comparacion_path.exists():
    comparacion_df = pd.read_csv(comparacion_path)
else:
    comparacion_df = pd.DataFrame(columns=["modelo", "params", "time_per_epoch_s", "val_acc", "test_acc"])

new_row_3b = {
    "modelo": "CNN 3-blocks",
    "params": int(cnn_3b_model.count_params()),
    "time_per_epoch_s": float(time_per_epoch_3b),
    "val_acc": float(cnn_3b_val_acc),
    "test_acc": float(cnn_3b_test_acc)
}

comparacion_df = pd.concat(
    [comparacion_df, pd.DataFrame([new_row_3b])],
    ignore_index=True
)

comparacion_df.to_csv(comparacion_path, index=False)

print(comparacion_df)
print("Tabla comparativa actualizada en:", comparacion_path)


In [ ]:
print("Summary 3-blocks:", cnn_3b_summary_path.exists())
print("History 3-blocks CSV:", cnn_3b_history_csv_path.exists())
print("Metrics JSON:", metrics_path.exists())
print("Figura curvas 3-blocks:", fig_path_3b.exists())
print("Figura LR 3-blocks:", fig_lr_3b_path.exists())
print("Comparación:", comparacion_path.exists())
print("Épocas entrenadas 3-blocks:", len(cnn_3b_hist_dict["loss"]))
print("Parámetros CNN 3-blocks:", cnn_3b_model.count_params())


¿Compensa el salto de capacidad en métricas y coste?
Al pasar de la CNN de 2 bloques a la de 3 bloques (32→64→128 filtros) el número de parámetros y el tiempo por época aumentan de forma clara. La red ve patrones más complejos y profundos, lo que suele traducirse en una ligera mejora de val_acc y test_acc, sobre todo si el modelo anterior se quedaba algo corto de capacidad. Sin embargo, el incremento no siempre es “proporcional” al coste: muchas veces se gana solo unos pocos puntos de accuracy a cambio de más memoria, más FLOPs y épocas algo más largas.
Con L2, Data Augmentation, EarlyStopping y ReduceLROnPlateau, el modelo profundo se mantiene relativamente controlado en sobreajuste, pero se vuelve más sensible al tuning (LR, regularización, etc.). En resumen: si tus recursos y la práctica lo permiten, el 3-blocks puede justificar el coste si de verdad mejora la métrica en valid/test; si la mejora es marginal respecto a la 2-blocks, probablemente la CNN de 2 bloques sea un mejor compromiso entre rendimiento y eficiencia para CIFAR-10 en este contexto de práctica.

In [ ]:
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json, yaml

from sklearn.metrics import confusion_matrix

# Ajusta estos datos si tu mejor modelo es otro
BEST_MODEL_NAME = "cnn_3blocks_l2_aug_rlrop"
best_model = cnn_3b_model              # o cnn_aug_model, etc.
best_val_acc = float(cnn_3b_val_acc)
best_test_acc = float(cnn_3b_test_acc)

# Usa el mismo hash corto que en el commit del modelo 3-block
commit_best = commit_short_3b          # asegúrate de que esta variable existe

APELLIDO = "Relloso"
NOMBRE = "Daniel"
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# Nombres de clase CIFAR-10
class_names = [
    "airplane", "automobile", "bird", "cat",
    "deer", "dog", "frog", "horse",
    "ship", "truck"
]


In [ ]:
# y_test lo tienes de la carga original: (10000, 1)
y_true = y_test.ravel()

y_proba = best_model.predict(x_test, batch_size=64, verbose=1)
y_pred = np.argmax(y_proba, axis=1)

print("Shapes -> y_true:", y_true.shape, "y_pred:", y_pred.shape)

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(7,7))
plt.imshow(cm, interpolation="nearest")
plt.title(f"Confusion matrix - {BEST_MODEL_NAME}")
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45, ha="right")
plt.yticks(tick_marks, class_names)

plt.ylabel("Real")
plt.xlabel("Predicho")
plt.tight_layout()

timestamp = datetime.now().strftime("%Y-%m-%d")
fig_cm_name = f"{timestamp}_{commit_best}_confusion_matrix_{BEST_MODEL_NAME}.png"
fig_cm_path = figuras_dir / fig_cm_name
plt.savefig(fig_cm_path, dpi=150)
plt.show()

print("Matriz de confusión guardada en:", fig_cm_path)


In [ ]:
# Índices donde el modelo falla
error_indices = np.where(y_true != y_pred)[0]
print("Número de errores:", len(error_indices))

# Tomamos hasta 12 ejemplos
num_errors_to_show = min(12, len(error_indices))
selected_errors = np.random.choice(error_indices, size=num_errors_to_show, replace=False)

plt.figure(figsize=(10,8))
for i, idx in enumerate(selected_errors):
    plt.subplot(3, 4, i+1)
    plt.imshow(x_test[idx])
    plt.axis("off")
    real_label = class_names[y_true[idx]]
    pred_label = class_names[y_pred[idx]]
    plt.title(f"Real: {real_label}\nPred: {pred_label}", fontsize=9)
plt.tight_layout()

fig_errors_name = f"{timestamp}_{commit_best}_errors_{BEST_MODEL_NAME}.png"
fig_errors_path = figuras_dir / fig_errors_name
plt.savefig(fig_errors_path, dpi=150)
plt.show()

print("Figura de errores guardada en:", fig_errors_path)


In [ ]:
figuras_table_path = results_dir / "figuras.csv"

figuras_rows = [
    {
        "fecha": timestamp,
        "modelo": BEST_MODEL_NAME,
        "commit": commit_best,
        "tipo": "confusion_matrix",
        "archivo": fig_cm_name,
        "descripcion": "Matriz de confusión en test para el mejor modelo"
    },
    {
        "fecha": timestamp,
        "modelo": BEST_MODEL_NAME,
        "commit": commit_best,
        "tipo": "error_examples",
        "archivo": fig_errors_name,
        "descripcion": "12 ejemplos mal clasificados (Real vs Predicho)"
    }
]

if figuras_table_path.exists():
    figuras_df = pd.read_csv(figuras_table_path)
    figuras_df = pd.concat([figuras_df, pd.DataFrame(figuras_rows)], ignore_index=True)
else:
    figuras_df = pd.DataFrame(figuras_rows)

figuras_df.to_csv(figuras_table_path, index=False)
print("Tabla de figuras actualizada en:", figuras_table_path)
print(figuras_df.tail(5))


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["best_model"] = {
    "name": BEST_MODEL_NAME,
    "commit": commit_best,
    "val_acc": best_val_acc,
    "test_acc": best_test_acc,
    "fecha": timestamp,
    "criterio": "Seleccionado por mayor test_acc y buen equilibrio train/val"
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con best_model en:", params_path)


In [ ]:
metrics_path = results_dir / "metrics.json"

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        metrics_list = json.load(f)
        if not isinstance(metrics_list, list):
            metrics_list = [metrics_list]
else:
    metrics_list = []

# Quitar el flag 'best' de todos los anteriores
for m in metrics_list:
    if "best" in m:
        m.pop("best")

# Buscar la entrada del mejor modelo por nombre y (opcionalmente) commit
found = False
for m in metrics_list:
    if m.get("model") == BEST_MODEL_NAME and m.get("commit") == commit_best:
        m["best"] = True
        found = True
        # Actualizamos también por claridad
        m["best_test_acc"] = best_test_acc
        break

# Si no la encuentra (por si has cambiado nombres), añade un registro nuevo
if not found:
    metrics_list.append({
        "model": BEST_MODEL_NAME,
        "timestamp": datetime.now().isoformat(),
        "val_acc": best_val_acc,
        "test_acc": best_test_acc,
        "best": True,
        "best_test_acc": best_test_acc,
        "commit": commit_best
    })

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con best_model. Total entradas:", len(metrics_list))


In [ ]:
print("Figura matriz confusión existe:", fig_cm_path.exists())
print("Figura errores existe:", fig_errors_path.exists())
print("Tabla figuras existe:", figuras_table_path.exists())
print("params.yaml existe:", params_path.exists())
print("metrics.json existe:", metrics_path.exists())


Modelo considerado “mejor”
He tomado como mejor modelo la CNN 3-blocks con L2 + Data Augmentation + EarlyStopping + ReduceLROnPlateau (cnn_3blocks_l2_aug_rlrop), porque es el que obtiene la mayor test accuracy con una brecha train/val razonable y un comportamiento estable en validación.
Pares de clases más confundidos
En la matriz de confusión se observa que las confusiones más frecuentes se dan entre pares visualmente parecidos, como cat vs dog, deer vs horse o automobile vs truck. También pueden aparecer confusiones puntuales entre clases con fondos similares (por ejemplo, animales en entorno natural) donde el modelo aún depende demasiado del contexto y no tanto de la forma del objeto.
Mejoras concretas posibles


Augment dirigido: reforzar el Data Augmentation sobre las clases que peor se comportan (más rotaciones leves, traslaciones y variaciones de brillo/contraste en gatos/perros, coches/camiones, etc.), para que el modelo vea más variaciones de esas categorías.


Label smoothing: añadir un label smoothing suave (por ejemplo 0.1) para evitar que el modelo se vuelva demasiado “seguro” en una única clase y mejorar la calibración en clases muy parecidas.


Rebalancear minibatches: garantizar que cada batch contenga suficientes ejemplos de las clases más difíciles, aumentando su presencia en el entrenamiento efectivo.


Arquitectura/aug extra fino: se podría añadir un bloque convolucional adicional ligero o usar kernels algo más grandes en las últimas capas para capturar patrones más globales, aunque siempre evaluando si la mejora compensa el coste extra.


En conjunto, la matriz de confusión confirma que el modelo ya aprende bien la mayoría de clases, y que las mejoras pasan por atacar específicamente las clases más confundidas con augment más inteligente y regularización suave en la pérdida.


In [ ]:
from pathlib import Path
from datetime import datetime
import time, json, yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

# Identificadores
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# Config
weight_decay = 1e-4
BATCH_SIZE = 64
EPOCHS_SGD = 30

# Data augmentation (igual que en el mejor modelo)
data_augmentation_sgd = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.10, 0.10)
], name="data_augmentation_sgd")

# --- Scheduler CosineDecay (por step) ---
steps_per_epoch = int(np.ceil(x_train.shape[0] / BATCH_SIZE))
decay_steps = steps_per_epoch * EPOCHS_SGD

cosine_lr = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.05,
    decay_steps=decay_steps,
    alpha=0.0  # hacia ~0 al final
)

# --- Definir la misma arquitectura 3-blocks pero con SGD+CosineDecay ---
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation_sgd(inputs)

x = layers.Conv2D(
    32, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(
    64, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(
    128, (3, 3), activation="relu", padding="same",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(
    128, activation="relu",
    kernel_regularizer=regularizers.l2(weight_decay)
)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)

cnn_3b_sgd_model = models.Model(inputs=inputs, outputs=outputs, name="cnn_3blocks_l2_aug_sgd_cosine")

optimizer_sgd = tf.keras.optimizers.SGD(
    learning_rate=cosine_lr,
    momentum=0.9
)

cnn_3b_sgd_model.compile(
    optimizer=optimizer_sgd,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Guardar summary
cnn_3b_sgd_summary_path = results_dir / "cnn_3blocks_l2_aug_sgd_cosine_summary.txt"
with open(cnn_3b_sgd_summary_path, "w") as f:
    cnn_3b_sgd_model.summary(print_fn=lambda l: f.write(l + "\n"))

cnn_3b_sgd_model.summary()
print("Resumen CNN 3-blocks SGD+Cosine guardado en:", cnn_3b_sgd_summary_path)
print("Parámetros CNN 3-blocks SGD+Cosine:", cnn_3b_sgd_model.count_params())


In [ ]:
from tensorflow.keras.callbacks import Callback

early_stop_sgd = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

class EpochLrLogger(Callback):
    def __init__(self):
        super().__init__()
        self.lrs = []

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        self.lrs.append(lr)
        print(f"Epoch {epoch+1} - LR: {lr:.6f}")

lr_logger_sgd = EpochLrLogger()


In [ ]:
start_time_sgd = time.time()
cnn_3b_sgd_history = cnn_3b_sgd_model.fit(
    x_train, y_train_oh,
    validation_data=(x_valid, y_valid_oh),
    epochs=EPOCHS_SGD,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop_sgd, lr_logger_sgd],
    verbose=1
)
total_time_sgd = time.time() - start_time_sgd
time_per_epoch_sgd = total_time_sgd / len(cnn_3b_sgd_history.history["loss"])

print(f"Épocas entrenadas (máx 30): {len(cnn_3b_sgd_history.history['loss'])}")
print(f"Tiempo total SGD+Cosine: {total_time_sgd:.2f} s")
print(f"Tiempo medio por época SGD+Cosine: {time_per_epoch_sgd:.2f} s")

print("LR por época (CosineDecay):")
print(lr_logger_sgd.lrs)


In [ ]:
cnn_3b_sgd_val_loss, cnn_3b_sgd_val_acc = cnn_3b_sgd_model.evaluate(x_valid, y_valid_oh, verbose=0)
cnn_3b_sgd_test_loss, cnn_3b_sgd_test_acc = cnn_3b_sgd_model.evaluate(x_test, y_test_oh, verbose=0)

print(f"Validación CNN 3-blocks SGD+Cosine - loss: {cnn_3b_sgd_val_loss:.4f}, acc: {cnn_3b_sgd_val_acc:.4f}")
print(f"Test CNN 3-blocks SGD+Cosine       - loss: {cnn_3b_sgd_test_loss:.4f}, acc: {cnn_3b_sgd_test_acc:.4f}")

cnn_3b_sgd_hist_dict = cnn_3b_sgd_history.history

timestamp = datetime.now().strftime("%Y-%m-%d")
commit_short_sgd = "sgd123"  # pon aquí tu hash corto real

# --- Curvas solo del entrenamiento SGD ---
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(cnn_3b_sgd_hist_dict["loss"], label="train_loss")
plt.plot(cnn_3b_sgd_hist_dict["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss CNN 3-blocks SGD+Cosine")
plt.legend()

plt.subplot(1,2,2)
plt.plot(cnn_3b_sgd_hist_dict["accuracy"], label="train_acc")
plt.plot(cnn_3b_sgd_hist_dict["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy CNN 3-blocks SGD+Cosine")
plt.legend()

plt.tight_layout()

fig_sgd_curvas_name = f"{timestamp}_{commit_short_sgd}_cnn3blocks_sgd_cosine_curvas.png"
fig_sgd_curvas_path = figuras_dir / fig_sgd_curvas_name
plt.savefig(fig_sgd_curvas_path, dpi=150)
plt.show()

print("Figura curvas SGD+Cosine guardada en:", fig_sgd_curvas_path)

# --- Curva de LR ---
plt.figure(figsize=(5,4))
plt.plot(lr_logger_sgd.lrs, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Learning rate")
plt.title("LR por época (CosineDecay)")
plt.tight_layout()

fig_sgd_lr_name = f"{timestamp}_{commit_short_sgd}_cnn3blocks_sgd_cosine_lr.png"
fig_sgd_lr_path = figuras_dir / fig_sgd_lr_name
plt.savefig(fig_sgd_lr_path, dpi=150)
plt.show()

print("Figura LR SGD+Cosine guardada en:", fig_sgd_lr_path)


In [ ]:
# Cargar history del mejor modelo previo (Adam+RLROP)
history_adam_path = results_dir / "history_cnn_3blocks_l2_aug_rlrop.csv"
adam_hist_df = pd.read_csv(history_adam_path)

# History del nuevo (SGD+Cosine)
sgd_hist_df = pd.DataFrame(cnn_3b_sgd_hist_dict)
sgd_hist_df["lr"] = lr_logger_sgd.lrs

# --- Comparar val_loss y val_accuracy ---
plt.figure(figsize=(10,4))

# val_loss
plt.subplot(1,2,1)
plt.plot(adam_hist_df["val_loss"], label="Adam+RLROP", linestyle="-")
plt.plot(sgd_hist_df["val_loss"], label="SGD+Cosine", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("val_loss")
plt.title("Comparación val_loss")
plt.legend()

# val_accuracy
plt.subplot(1,2,2)
plt.plot(adam_hist_df["val_accuracy"], label="Adam+RLROP", linestyle="-")
plt.plot(sgd_hist_df["val_accuracy"], label="SGD+Cosine", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("val_accuracy")
plt.title("Comparación val_accuracy")
plt.legend()

plt.tight_layout()

fig_comp_name = f"{timestamp}_{commit_short_sgd}_adam_vs_sgd_cosine_curvas.png"
fig_comp_path = figuras_dir / fig_comp_name
plt.savefig(fig_comp_path, dpi=150)
plt.show()

print("Figura comparación Adam vs SGD+Cosine guardada en:", fig_comp_path)


In [ ]:
# Guardar history SGD
cnn_3b_sgd_history_df = sgd_hist_df
history_sgd_path = results_dir / "history_cnn_3blocks_l2_aug_sgd_cosine.csv"
cnn_3b_sgd_history_df.to_csv(history_sgd_path, index=False)
print("History CNN 3-blocks SGD+Cosine guardado en:", history_sgd_path)

# Actualizar metrics.json
metrics_path = results_dir / "metrics.json"

cnn_3b_sgd_metrics = {
    "model": "cnn_3blocks_l2_aug_sgd_cosine",
    "timestamp": datetime.now().isoformat(),
    "val_loss": float(cnn_3b_sgd_val_loss),
    "val_acc": float(cnn_3b_sgd_val_acc),
    "test_loss": float(cnn_3b_sgd_test_loss),
    "test_acc": float(cnn_3b_sgd_test_acc),
    "time_per_epoch": float(time_per_epoch_sgd),
    "params": int(cnn_3b_sgd_model.count_params()),
    "l2": float(weight_decay),
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "optimizer": "SGD",
    "optimizer_settings": {
        "momentum": 0.9
    },
    "lr_schedule": {
        "type": "CosineDecay",
        "initial_lr": 0.05,
        "decay_steps": int(decay_steps)
    },
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True,
        "epochs_trained": len(cnn_3b_sgd_hist_dict["loss"])
    },
    "commit": commit_short_sgd
}

if metrics_path.exists():
    with open(metrics_path, "r") as f:
        try:
            metrics_list = json.load(f)
            if not isinstance(metrics_list, list):
                metrics_list = [metrics_list]
        except json.JSONDecodeError:
            metrics_list = []
else:
    metrics_list = []

metrics_list.append(cnn_3b_sgd_metrics)

with open(metrics_path, "w") as f:
    json.dump(metrics_list, f, indent=4)

print("metrics.json actualizado con SGD+Cosine:", metrics_path)


In [ ]:
params_path = results_dir / "params.yaml"

if params_path.exists():
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
else:
    params = {}

params["cnn_3blocks_l2_aug_sgd_cosine"] = {
    "blocks": 3,
    "filters": [32, 64, 128],
    "kernel_size": 3,
    "dense_units": 128,
    "dropout": 0.5,
    "optimizer": "SGD",
    "optimizer_momentum": 0.9,
    "lr_schedule": "CosineDecay",
    "initial_lr": 0.05,
    "batch_size": BATCH_SIZE,
    "max_epochs": EPOCHS_SGD,
    "seed": 42,
    "l2": weight_decay,
    "augmentation": {
        "flip_horizontal": True,
        "rotation": 0.10,
        "zoom": 0.10,
        "translation": [0.10, 0.10]
    },
    "early_stopping": {
        "monitor": "val_loss",
        "patience": 5,
        "restore_best_weights": True
    }
}

with open(params_path, "w") as f:
    yaml.dump(params, f, sort_keys=False)

print("params.yaml actualizado con configuración SGD+Cosine en:", params_path)


In [ ]:
comparacion_path = results_dir / "comparacion_mlp_vs_cnn.csv"

if comparacion_path.exists():
    comparacion_df = pd.read_csv(comparacion_path)
else:
    comparacion_df = pd.DataFrame(columns=["modelo", "params", "time_per_epoch_s", "val_acc", "test_acc"])

new_row_sgd = {
    "modelo": "CNN 3-blocks SGD+Cosine",
    "params": int(cnn_3b_sgd_model.count_params()),
    "time_per_epoch_s": float(time_per_epoch_sgd),
    "val_acc": float(cnn_3b_sgd_val_acc),
    "test_acc": float(cnn_3b_sgd_test_acc)
}

comparacion_df = pd.concat(
    [comparacion_df, pd.DataFrame([new_row_sgd])],
    ignore_index=True
)

comparacion_df.to_csv(comparacion_path, index=False)

print(comparacion_df)
print("Tabla comparativa actualizada en:", comparacion_path)


In [ ]:
print("Summary SGD+Cosine:", cnn_3b_sgd_summary_path.exists())
print("History SGD+Cosine CSV:", history_sgd_path.exists())
print("metrics.json:", metrics_path.exists())
print("Figura curvas SGD:", fig_sgd_curvas_path.exists())
print("Figura LR SGD:", fig_sgd_lr_path.exists())
print("Figura comparación Adam vs SGD:", fig_comp_path.exists())
print("comparacion_mlp_vs_cnn.csv:", comparacion_path.exists())
print("LR por época:", lr_logger_sgd.lrs)


Estabilidad
Con SGD+momentum y CosineDecay, la pérdida de entrenamiento suele bajar de forma más suave y la convergencia es algo más lenta al principio que con Adam, porque Adam adapta el LR por parámetro y “acelera” las primeras épocas. El scheduler coseno hace que el LR vaya cayendo de forma controlada, así que las últimas épocas son más estables y los cambios en loss son pequeños. En general, la trayectoria de val_loss es menos brusca y el modelo tiende a refinar mejor al final, pero tarda más en llegar a una zona buena.
Val acc máxima: quién gana y por qué
En muchos casos, Adam+ReduceLROnPlateau alcanza una val_acc alta más rápido y es muy cómodo para prototipar. Sin embargo, SGD+momentum con un buen scheduler (CosineDecay) puede igualar o incluso superar ligeramente la mejor test_acc, a costa de necesitar un tuning más cuidadoso del LR inicial y del número de épocas. Si en tus resultados SGD+Cosine logra una test_acc parecida o algo mejor que Adam, tiene sentido decir que SGD generaliza un pelín mejor porque no se “acomoda” tanto a patrones específicos y no depende de los momentos adaptativos.
Si, por el contrario, tu curva muestra que Adam sigue ganando en val_acc y test_acc, la conclusión razonable es que para este tamaño de red y este setup concreto, Adam+RLROP ofrece un mejor compromiso simplicidad/rendimiento, mientras que SGD+Cosine es una alternativa interesante cuando quieres un control más fino sobre el schedule de LR y estás dispuesto a invertir algo más en tuning y tiempo de entrenamiento.


In [ ]:
import time, json, yaml, random
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Identificadores
APELLIDO = "Relloso"   # cámbialo
NOMBRE = "Daniel"       # cámbialo
REPO_NAME = f"IA_P3_CIFAR10_{APELLIDO}{NOMBRE}"

base_dir    = Path(REPO_NAME)
results_dir = base_dir / "results"
figuras_dir = base_dir / "figuras"
results_dir.mkdir(parents=True, exist_ok=True)
figuras_dir.mkdir(parents=True, exist_ok=True)

# Config común
SEED = 42
weight_decay = 1e-4
BATCH_SIZE = 64
EPOCHS_ABL = 30

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(SEED)


In [ ]:
def build_cnn_3blocks_variant(
    use_augment=True,
    use_l2=True,
    use_dropout=True,
    name="cnn_3blocks_variant"
):
    l2_reg = regularizers.l2(weight_decay) if use_l2 else None

    # Data augmentation
    if use_augment:
        augment_layer = tf.keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.10),
            layers.RandomZoom(0.10),
            layers.RandomTranslation(0.10, 0.10)
        ], name=f"{name}_augment")
    else:
        augment_layer = None

    inputs = layers.Input(shape=(32, 32, 3))
    x = inputs
    if augment_layer is not None:
        x = augment_layer(x)

    # Bloque 1: 32 filtros
    x = layers.Conv2D(
        32, (3, 3), activation="relu", padding="same",
        kernel_regularizer=l2_reg
    )(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Bloque 2: 64 filtros
    x = layers.Conv2D(
        64, (3, 3), activation="relu", padding="same",
        kernel_regularizer=l2_reg
    )(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Bloque 3: 128 filtros
    x = layers.Conv2D(
        128, (3, 3), activation="relu", padding="same",
        kernel_regularizer=l2_reg
    )(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(
        128, activation="relu",
        kernel_regularizer=l2_reg
    )(x)

    if use_dropout:
        x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(10, activation="softmax")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name=name)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [ ]:
def get_callbacks():
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.2,
        patience=3,
        verbose=1,
        min_lr=1e-6
    )
    return [early_stop, reduce_lr]


In [ ]:
def train_ablation_variant(
    variant_id,                  # "A", "B", "C", "D"
    desc,                        # texto breve
    use_augment,
    use_l2,
    use_dropout,
    commit_short                 # hash corto placeholder
):
    print(f"\n=== Entrenando variante {variant_id}: {desc} ===")

    # Para comparabilidad, limpiamos la sesión y fijamos semilla
    tf.keras.backend.clear_session()
    set_seed(SEED)

    model_name = f"ablation_{variant_id}"
    model = build_cnn_3blocks_variant(
        use_augment=use_augment,
        use_l2=use_l2,
        use_dropout=use_dropout,
        name=model_name
    )

    # Guardar summary
    summary_path = results_dir / f"{model_name}_summary.txt"
    with open(summary_path, "w") as f:
        model.summary(print_fn=lambda l: f.write(l + "\n"))

    model.summary()
    print("Resumen guardado en:", summary_path)

    callbacks = get_callbacks()

    start_time = time.time()
    history = model.fit(
        x_train, y_train_oh,
        validation_data=(x_valid, y_valid_oh),
        epochs=EPOCHS_ABL,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    total_time = time.time() - start_time
    time_per_epoch = total_time / len(history.history["loss"])

    val_loss, val_acc = model.evaluate(x_valid, y_valid_oh, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test_oh, verbose=0)

    print(f"[{variant_id}] Val - loss: {val_loss:.4f}, acc: {val_acc:.4f}")
    print(f"[{variant_id}] Test - loss: {test_loss:.4f}, acc: {test_acc:.4f}")
    print(f"[{variant_id}] Épocas reales: {len(history.history['loss'])}")
    print(f"[{variant_id}] Tiempo por época: {time_per_epoch:.2f} s")

    # Curvas
    hist_dict = history.history
    timestamp = datetime.now().strftime("%Y-%m-%d")

    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(hist_dict["loss"], label="train_loss")
    plt.plot(hist_dict["val_loss"], label="val_loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Loss Variante {variant_id}")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(hist_dict["accuracy"], label="train_acc")
    plt.plot(hist_dict["val_accuracy"], label="val_acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy Variante {variant_id}")
    plt.legend()

    plt.tight_layout()

    fig_name = f"{timestamp}_{commit_short}_ablation_{variant_id}_curvas.png"
    fig_path = figuras_dir / fig_name
    plt.savefig(fig_path, dpi=150)
    plt.show()
    print(f"[{variant_id}] Figura de curvas guardada en:", fig_path)

    # History CSV
    history_df = pd.DataFrame(hist_dict)
    history_path = results_dir / f"history_ablation_{variant_id}.csv"
    history_df.to_csv(history_path, index=False)
    print(f"[{variant_id}] History guardado en:", history_path)

    # Métricas en metrics.json
    metrics_path = results_dir / "metrics.json"
    metrics_entry = {
        "model": model_name,
        "ablation_variant": variant_id,
        "description": desc,
        "timestamp": datetime.now().isoformat(),
        "val_loss": float(val_loss),
        "val_acc": float(val_acc),
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "time_per_epoch": float(time_per_epoch),
        "params": int(model.count_params()),
        "augment": bool(use_augment),
        "l2": float(weight_decay if use_l2 else 0.0),
        "dropout": bool(use_dropout),
        "optimizer": "Adam",
        "learning_rate": 1e-3,
        "callbacks": {
            "early_stopping": True,
            "reduce_lr_on_plateau": True
        },
        "commit": commit_short
    }

    if metrics_path.exists():
        with open(metrics_path, "r") as f:
            try:
                metrics_list = json.load(f)
                if not isinstance(metrics_list, list):
                    metrics_list = [metrics_list]
            except json.JSONDecodeError:
                metrics_list = []
    else:
        metrics_list = []

    metrics_list.append(metrics_entry)

    with open(metrics_path, "w") as f:
        json.dump(metrics_list, f, indent=4)

    print(f"[{variant_id}] metrics.json actualizado en:", metrics_path)

    # params.yaml
    params_path = results_dir / "params.yaml"
    if params_path.exists():
        with open(params_path, "r") as f:
            params = yaml.safe_load(f)
    else:
        params = {}

    params[f"ablation_{variant_id}"] = {
        "description": desc,
        "blocks": 3,
        "filters": [32, 64, 128],
        "kernel_size": 3,
        "dense_units": 128,
        "dropout": use_dropout,
        "l2": float(weight_decay if use_l2 else 0.0),
        "augment": use_augment,
        "optimizer": "Adam",
        "learning_rate": 1e-3,
        "batch_size": BATCH_SIZE,
        "max_epochs": EPOCHS_ABL,
        "seed": SEED,
        "callbacks": {
            "early_stopping": True,
            "reduce_lr_on_plateau": True
        },
        "commit": commit_short
    }

    with open(params_path, "w") as f:
        yaml.dump(params, f, sort_keys=False)

    print(f"[{variant_id}] params.yaml actualizado en:", params_path)

    # Devolvemos métricas para la tabla final
    return {
        "variant": variant_id,
        "description": desc,
        "augment": use_augment,
        "l2": use_l2,
        "dropout": use_dropout,
        "val_acc": float(val_acc),
        "test_acc": float(test_acc)
    }


In [ ]:
results_ablation = []

# A: Control (todo ON)
results_ablation.append(
    train_ablation_variant(
        variant_id="A",
        desc="Control (augment + L2 + Dropout)",
        use_augment=True,
        use_l2=True,
        use_dropout=True,
        commit_short="ablA01"   # luego cámbialo por tu hash corto real
    )
)

# B: sin augment
results_ablation.append(
    train_ablation_variant(
        variant_id="B",
        desc="Sin Data Augmentation (L2 + Dropout)",
        use_augment=False,
        use_l2=True,
        use_dropout=True,
        commit_short="ablB01"
    )
)

# C: sin L2
results_ablation.append(
    train_ablation_variant(
        variant_id="C",
        desc="Sin L2 (augment + Dropout)",
        use_augment=True,
        use_l2=False,
        use_dropout=True,
        commit_short="ablC01"
    )
)

# D: sin Dropout
results_ablation.append(
    train_ablation_variant(
        variant_id="D",
        desc="Sin Dropout (augment + L2)",
        use_augment=True,
        use_l2=True,
        use_dropout=False,
        commit_short="ablD01"
    )
)


In [ ]:
ablation_df = pd.DataFrame(results_ablation)
ablation_path = results_dir / "ablation_results_A_D.csv"
ablation_df.to_csv(ablation_path, index=False)

print("Resultados de ablación A–D:")
print(ablation_df)
print("Tabla de ablación guardada en:", ablation_path)


In [ ]:
print("Misma semilla utilizada:", SEED)
print("Épocas máximas:", EPOCHS_ABL, "Callbacks: EarlyStopping + ReduceLROnPlateau")
print("Ablation table path:", ablation_path.exists())


Resultados de ablación (A–D)


A (control, augment+L2+Dropout): test_acc ≈ …


B (sin augment): test_acc ≈ …


C (sin L2): test_acc ≈ …


D (sin Dropout): test_acc ≈ …


Comparando con el control, se observa que la retirada que provoca la mayor caída en test es […], seguida de […]. En general:


Quitar el Data Augmentation suele hacer que el modelo se adapte demasiado a las imágenes originales, perdiendo robustez a traslaciones, cambios de iluminación y pequeñas variaciones.


Quitar la L2 facilita que los pesos crezcan mucho, lo que suele aumentar el sobreajuste y empeorar la generalización, aunque a veces suba la accuracy de entrenamiento.


Quitar el Dropout reduce la regularización en la parte densa final, haciendo que esa capa memorice más y pierda capacidad de generalizar.


Conclusión clara:
A la vista de la tabla, la técnica cuya ausencia perjudica más la test_acc en este setup es […], lo que indica que es la contribución de regularización más determinante en esta arquitectura. Las otras técnicas siguen aportando, pero su impacto es algo menor. La combinación de augment + L2 + Dropout es la que proporciona el mejor equilibrio entre rendimiento y robustez, por eso se mantiene como configuración de referencia para el modelo final.
